<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


In [ ]:
## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [21]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

In [22]:
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


Now, you can take a look at what are the coordinates for each site.


In [5]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [6]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [7]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [14]:
# Initial the map

# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
import folium

# Assume nasa_coordinate is the average location of all launch sites
nasa_coordinate = [launch_sites_df['Lat'].mean(), launch_sites_df['Long'].mean()]

# Initialize the map centered around nasa_coordinate
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# Iterate over each launch site and add a Circle and Marker
for index, row in launch_sites_df.iterrows():
    # Add a circle for the launch site
    folium.Circle(
        location=[row['Lat'], row['Long']],
        radius=1000,  # Adjust radius as needed
        color='#000000',
        fill=True,
        fill_color='#3186cc'
    ).add_child(folium.Popup(row['Launch Site'])).add_to(site_map)
    
    # Add a marker for the launch site
    folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12; color:#d35400;"><b>{row["Launch Site"]}</b></div>'
        )
    ).add_to(site_map)

# Display the map
site_map



The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [23]:
# Task 2: Mark the success/failed launches for each site on the map
# Initialize the map centered around NASA coordinates
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Iterate over each launch site and add a Circle and Marker
for index, row in spacex_df.iterrows():
    # Add a circle for the launch site
    folium.Circle(
        location=[row['Lat'], row['Long']],
        radius=1000,
        color='#000000',
        fill=True,
        fill_color='#3186cc'
    ).add_child(folium.Popup(row['Launch Site'])).add_to(site_map)
    
    # Add a marker for the launch site
    folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12; color:#d35400;"><b>{row["Launch Site"]}</b></div>'
        )
    ).add_to(site_map)

# Now, add success/failure markers
for index, row in spacex_df.iterrows():
    # Define the color based on the success or failure
    color = 'green' if row['class'] == 1 else 'red'
    
    # Add the marker
    folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=folium.Icon(color=color),
        popup=f"Flight {row['Flight Number']}: {'Success' if row['class'] == 1 else 'Failure'}"
    ).add_to(site_map)

# Display the map
site_map



Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [24]:
spacex_df.tail(10)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
46,43,2017-10-11,22:53:00,F9 FT B1031.2,KSC LC-39A,SES-11 / EchoStar 105,5200.00,GTO,SES EchoStar,Success (drone ship),1,28.573255,-80.646895
47,44,2017-10-30,19:34:00,F9 B4 B1042.1,KSC LC-39A,Koreasat 5A,3500.00,GTO,KT Corporation,Success (drone ship),1,28.573255,-80.646895
48,54,2018-05-11,20:14:00,F9 B5 B1046.1,KSC LC-39A,Bangabandhu-1,3600.00,GTO,Thales-Alenia/BTRC,Success (drone ship),1,28.573255,-80.646895
49,45,2017-12-15,15:36:00,F9 FT B1035.2,CCAFS SLC-40,SpaceX CRS-13,2205.00,LEO (ISS),NASA (CRS),Success (ground pad),1,28.563197,-80.576820
50,47,2018-01-08,1:00:00,F9 B4 B1043.1,CCAFS SLC-40,Zuma,3696.65,LEO,Northrop Grumman,Success (ground pad),1,28.563197,-80.576820
51,48,2018-01-31,21:25:00,F9 FT B1032.2,CCAFS SLC-40,GovSat-1 / SES-16,4230.00,GTO,SES,Controlled (ocean),0,28.563197,-80.576820
52,50,2018-03-06,5:33:00,F9 B4 B1044,CCAFS SLC-40,Hispasat 30W-6 PODSat,6092.00,GTO,Hispasat NovaWurks,No attempt,0,28.563197,-80.576820
53,52,2018-04-02,20:30:00,F9 B4 B1039.2,CCAFS SLC-40,SpaceX CRS-14,2647.00,LEO (ISS),NASA (CRS),No attempt,0,28.563197,-80.576820
54,53,2018-04-18,22:51:00,F9 B4 B1045.1,CCAFS SLC-40,Transiting Exoplanet Survey Satellite (TESS),362.00,HEO,NASA (LSP),Success (drone ship),1,28.563197,-80.576820
55,56,2018-06-04,4:45:00,F9 B4 B1040.2,CCAFS SLC-40,SES-12,5384.00,GTO,SES,No attempt,0,28.563197,-80.576820


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [25]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [28]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be redimport pandas as pd

# Sample DataFrame
data = {
    'Flight Number': [43, 44, 54, 45, 47, 48, 50, 52, 53, 56],
    'Date': ['2017-10-11', '2017-10-30', '2018-05-11', '2017-12-15', '2018-01-08', '2018-01-31', '2018-03-06', '2018-04-02', '2018-04-18', '2018-06-04'],
    'Time (UTC)': ['22:53:00', '19:34:00', '20:14:00', '15:36:00', '1:00:00', '21:25:00', '5:33:00', '20:30:00', '22:51:00', '4:45:00'],
    'Booster Version': ['F9 FT B1031.2', 'F9 B4 B1042.1', 'F9 B5 B1046.1', 'F9 FT B1035.2', 'F9 B4 B1043.1', 'F9 FT B1032.2', 'F9 B4 B1044', 'F9 B4 B1039.2', 'F9 B4 B1045.1', 'F9 B4 B1040.2'],
    'Launch Site': ['KSC LC-39A', 'KSC LC-39A', 'KSC LC-39A', 'CCAFS SLC-40', 'CCAFS SLC-40', 'CCAFS SLC-40', 'CCAFS SLC-40', 'CCAFS SLC-40', 'CCAFS SLC-40', 'CCAFS SLC-40'],
    'Payload': ['SES-11 / EchoStar 105', 'Koreasat 5A', 'Bangabandhu-1', 'SpaceX CRS-13', 'Zuma', 'GovSat-1 / SES-16', 'Hispasat 30W-6 PODSat', 'SpaceX CRS-14', 'Transiting Exoplanet Survey Satellite (TESS)', 'SES-12'],
    'Payload Mass (kg)': [5200.00, 3500.00, 3600.00, 2205.00, 3696.65, 4230.00, 6092.00, 2647.00, 362.00, 5384.00],
    'Orbit': ['GTO', 'GTO', 'GTO', 'LEO (ISS)', 'LEO', 'GTO', 'GTO', 'LEO (ISS)', 'HEO', 'GTO'],
    'Customer': ['SES EchoStar', 'KT Corporation', 'Thales-Alenia/BTRC', 'NASA (CRS)', 'Northrop Grumman', 'SES', 'Hispasat NovaWurks', 'NASA (CRS)', 'NASA (LSP)', 'SES'],
    'Landing Outcome': ['Success (drone ship)', 'Success (drone ship)', 'Success (drone ship)', 'Success (ground pad)', 'Success (ground pad)', 'Controlled (ocean)', 'No attempt', 'No attempt', 'Success (drone ship)', 'No attempt'],
    'class': [1, 1, 1, 1, 1, 0, 0, 0, 1, 0],
    'Lat': [28.573255, 28.573255, 28.573255, 28.563197, 28.563197, 28.563197, 28.563197, 28.563197, 28.563197, 28.563197],
    'Long': [-80.646895, -80.646895, -80.646895, -80.576820, -80.576820, -80.576820, -80.576820, -80.576820, -80.576820, -80.576820]
}

df = pd.DataFrame(data)

# Define a function to determine marker color based on the class value
def determine_marker_color(cls):
    return 'green' if cls == 1 else 'red'

# Apply the function to create the marker_color column
df['marker_color'] = df['class'].apply(determine_marker_color)

# Display the DataFrame to check the result
print(df.head())

# Define the initial map
nasa_coordinate = [28.563197, -80.576820]  # Center around Cape Canaveral
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

# Add a MarkerCluster to group markers
marker_cluster = MarkerCluster().add_to(site_map)

# Add Circles and Markers for each launch site
for idx, row in df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    color = row['marker_color']
    popup_text = f"Flight Number: {row['Flight Number']}<br>Payload: {row['Payload']}<br>Class: {'Success' if row['class'] == 1 else 'Failure'}"
    
    # Add Circle
    folium.Circle(
        coordinate,
        radius=1000,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.6
    ).add_to(marker_cluster)
    
    # Add Marker
    folium.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12px; color:{color};"><b>{row["Flight Number"]}</b></div>'
        ),
        popup=popup_text
    ).add_to(marker_cluster)

# Display the map
site_map.save('launch_sites_map.html')


   Flight Number        Date Time (UTC) Booster Version   Launch Site  \
0             43  2017-10-11   22:53:00   F9 FT B1031.2    KSC LC-39A   
1             44  2017-10-30   19:34:00   F9 B4 B1042.1    KSC LC-39A   
2             54  2018-05-11   20:14:00   F9 B5 B1046.1    KSC LC-39A   
3             45  2017-12-15   15:36:00   F9 FT B1035.2  CCAFS SLC-40   
4             47  2018-01-08    1:00:00   F9 B4 B1043.1  CCAFS SLC-40   

                 Payload  Payload Mass (kg)      Orbit            Customer  \
0  SES-11 / EchoStar 105            5200.00        GTO        SES EchoStar   
1            Koreasat 5A            3500.00        GTO      KT Corporation   
2          Bangabandhu-1            3600.00        GTO  Thales-Alenia/BTRC   
3          SpaceX CRS-13            2205.00  LEO (ISS)          NASA (CRS)   
4                   Zuma            3696.65        LEO    Northrop Grumman   

        Landing Outcome  class        Lat       Long marker_color  
0  Success (drone ship) 

*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [30]:
# Create and add 'marker_color' column based on 'class' value
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')


In [31]:
print(spacex_df.head())  # To inspect the first few rows and ensure 'marker_color' is present


   Flight Number        Date Time (UTC) Booster Version  Launch Site  \
0              1  2010-06-04   18:45:00  F9 v1.0  B0003  CCAFS LC-40   
1              2  2010-12-08   15:43:00  F9 v1.0  B0004  CCAFS LC-40   
2              3  2012-05-22    7:44:00  F9 v1.0  B0005  CCAFS LC-40   
3              4  2012-10-08    0:35:00  F9 v1.0  B0006  CCAFS LC-40   
4              5  2013-03-01   15:10:00  F9 v1.0  B0007  CCAFS LC-40   

                                             Payload  Payload Mass (kg)  \
0               Dragon Spacecraft Qualification Unit                0.0   
1  Dragon demo flight C1, two CubeSats,  barrel o...                0.0   
2                             Dragon demo flight C2+              525.0   
3                                       SpaceX CRS-1              500.0   
4                                       SpaceX CRS-2              677.0   

       Orbit         Customer        Landing Outcome  class        Lat  \
0        LEO           SpaceX  Failure   (

In [32]:
import folium
from folium.plugins import MarkerCluster
from folium.features import DivIcon

# Define the initial map
nasa_coordinate = [28.563197, -80.576820]  # Center around Cape Canaveral
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

# Add a MarkerCluster to group markers
marker_cluster = MarkerCluster().add_to(site_map)

# Add Circles and Markers for each launch site
for index, record in spacex_df.iterrows():
    coordinate = [record['Lat'], record['Long']]
    color = record['marker_color']
    popup_text = f"Flight Number: {record['Flight Number']}<br>Payload: {record['Payload']}<br>Class: {'Success' if record['class'] == 1 else 'Failure'}"
    
    # Add Circle
    folium.Circle(
        coordinate,
        radius=1000,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.6
    ).add_to(marker_cluster)
    
    # Add Marker
    marker = folium.Marker(
        location=coordinate,
        icon=folium.Icon(color=color, icon='info-sign'),  # Customize the icon color
        popup=popup_text
    )
    
    # Add Marker to MarkerCluster
    marker_cluster.add_child(marker)

# Display the map
site_map.save('launch_sites_map.html')


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [33]:
# TASK 3: Calculate the distances between a launch site to its proximities
import pandas as pd
import numpy as np

# Function to calculate distance using Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of Earth in km
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    
    a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return distance

# Sample DataFrame with launch sites
launch_sites_df = pd.DataFrame({
    'Launch Site': ['KSC LC-39A', 'CCAFS SLC-40'],
    'Lat': [28.573255, 28.563197],
    'Long': [-80.646895, -80.576820]
})

# Sample DataFrame with proximity points (for example, other launch sites)
proximities_df = pd.DataFrame({
    'Proximity': ['Site A', 'Site B'],
    'Lat': [28.580000, 28.570000],
    'Long': [-80.650000, -80.580000]
})

# Create a DataFrame to store distances
distances_df = pd.DataFrame()

# Calculate distances
for _, site_row in launch_sites_df.iterrows():
    site_name = site_row['Launch Site']
    site_lat = site_row['Lat']
    site_long = site_row['Long']
    
    distances = []
    for _, prox_row in proximities_df.iterrows():
        prox_name = prox_row['Proximity']
        prox_lat = prox_row['Lat']
        prox_long = prox_row['Long']
        distance = haversine(site_lat, site_long, prox_lat, prox_long)
        distances.append({'Launch Site': site_name, 'Proximity': prox_name, 'Distance (km)': distance})
    
    # Append results to the distances DataFrame
    distances_df = pd.concat([distances_df, pd.DataFrame(distances)], ignore_index=True)

# Display results
print(distances_df)



    Launch Site Proximity  Distance (km)
0    KSC LC-39A    Site A       0.808978
1    KSC LC-39A    Site B       6.542556
2  CCAFS SLC-40    Site A       7.386504
3  CCAFS SLC-40    Site B       0.817725


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [34]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [35]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [37]:
import pandas as pd
import numpy as np

# Function to calculate distance using Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of Earth in km
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    
    a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return distance

# Sample DataFrame with launch sites
launch_sites_df = pd.DataFrame({
    'Launch Site': ['KSC LC-39A', 'CCAFS SLC-40'],
    'Lat': [28.573255, 28.563197],
    'Long': [-80.646895, -80.576820]
})

# Sample DataFrame with coastline coordinates
coastline_df = pd.DataFrame({
    'Coastline': ['Coastline A', 'Coastline B'],
    'Lat': [28.580000, 28.570000],
    'Long': [-80.650000, -80.580000]
})

# Create a list to store results
results = []

# Find the closest coastline
for _, site_row in launch_sites_df.iterrows():
    site_name = site_row['Launch Site']
    site_lat = site_row['Lat']
    site_long = site_row['Long']
    
    min_distance = float('inf')
    closest_coastline = None
    closest_lat = None
    closest_long = None
    
    for _, coast_row in coastline_df.iterrows():
        coast_lat = coast_row['Lat']
        coast_long = coast_row['Long']
        distance = haversine(site_lat, site_long, coast_lat, coast_long)
        
        if distance < min_distance:
            min_distance = distance
            closest_coastline = coast_row['Coastline']
            closest_lat = coast_lat
            closest_long = coast_long
    
    # Append results to the list
    results.append({
        'Launch Site': site_name,
        'Closest Coastline': closest_coastline,
        'Distance (km)': min_distance,
        'Coastline Lat': closest_lat,
        'Coastline Long': closest_long
    })

# Convert the results list to a DataFrame
closest_coastline_df = pd.DataFrame(results)

# Display results
print(closest_coastline_df)


    Launch Site Closest Coastline  Distance (km)  Coastline Lat  \
0    KSC LC-39A       Coastline A       0.808978          28.58   
1  CCAFS SLC-40       Coastline B       0.817725          28.57   

   Coastline Long  
0          -80.65  
1          -80.58  


In [38]:
import folium
from folium.features import DivIcon

# Create the map centered on a general location (for example, KSC LC-39A)
site_map = folium.Map(location=[28.573255, -80.646895], zoom_start=10)

# Function to calculate distance using Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of Earth in km
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    
    a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return distance

# Sample DataFrame with launch sites
launch_sites_df = pd.DataFrame({
    'Launch Site': ['KSC LC-39A'],
    'Lat': [28.573255],
    'Long': [-80.646895]
})

# Sample DataFrame with coastline coordinates
coastline_df = pd.DataFrame({
    'Coastline': ['Coastline A'],
    'Lat': [28.580000],
    'Long': [-80.650000]
})

# Find the closest coastline
for _, site_row in launch_sites_df.iterrows():
    site_name = site_row['Launch Site']
    site_lat = site_row['Lat']
    site_long = site_row['Long']
    
    min_distance = float('inf')
    closest_coastline = None
    closest_lat = None
    closest_long = None
    
    for _, coast_row in coastline_df.iterrows():
        coast_lat = coast_row['Lat']
        coast_long = coast_row['Long']
        distance = haversine(site_lat, site_long, coast_lat, coast_long)
        
        if distance < min_distance:
            min_distance = distance
            closest_coastline = coast_row['Coastline']
            closest_lat = coast_lat
            closest_long = coast_long
    
    # Create a folium.Marker for the closest coastline
    distance_marker = folium.Marker(
        [closest_lat, closest_long],
        icon=DivIcon(
            icon_size=(150,50),
            icon_anchor=(0,0),
            html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(min_distance)
        )
    )
    
    # Add the marker to the map
    site_map.add_child(distance_marker)

# Display the map
site_map


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [39]:
import folium

# Initialize the map
site_map = folium.Map(location=[28.573255, -80.646895], zoom_start=10)

# Sample DataFrame with launch sites
launch_sites_df = pd.DataFrame({
    'Launch Site': ['KSC LC-39A'],
    'Lat': [28.573255],
    'Long': [-80.646895]
})

# Sample DataFrame with coastline coordinates
coastline_df = pd.DataFrame({
    'Coastline': ['Coastline A'],
    'Lat': [28.580000],
    'Long': [-80.650000]
})

# Find the closest coastline and create a PolyLine
for _, site_row in launch_sites_df.iterrows():
    site_name = site_row['Launch Site']
    site_lat = site_row['Lat']
    site_long = site_row['Long']
    
    min_distance = float('inf')
    closest_coastline = None
    closest_lat = None
    closest_long = None
    
    for _, coast_row in coastline_df.iterrows():
        coast_lat = coast_row['Lat']
        coast_long = coast_row['Long']
        distance = haversine(site_lat, site_long, coast_lat, coast_long)
        
        if distance < min_distance:
            min_distance = distance
            closest_coastline = coast_row['Coastline']
            closest_lat = coast_lat
            closest_long = coast_long
    
    # Coordinates for the PolyLine
    coordinates = [[site_lat, site_long], [closest_lat, closest_long]]
    
    # Create and add the PolyLine to the map
    polyline = folium.PolyLine(
        locations=coordinates,
        color='blue',
        weight=2,
        opacity=0.7
    )
    site_map.add_child(polyline)

# Display the map
site_map


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [40]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
import folium
from folium.plugins import MousePosition
from folium.features import DivIcon
from geopy.distance import geodesic as gd

# Initialize the map
site_map = folium.Map(location=[28.573255, -80.646895], zoom_start=10)

# Add MousePosition plugin to the map
mouse_position = MousePosition()
site_map.add_child(mouse_position)

# Sample DataFrame with launch sites
launch_sites_df = pd.DataFrame({
    'Launch Site': ['KSC LC-39A'],
    'Lat': [28.573255],
    'Long': [-80.646895]
})

# Sample DataFrame with cities, railways, highways, etc.
locations_df = pd.DataFrame({
    'Type': ['City', 'Railway', 'Highway'],
    'Name': ['Orlando', 'Railway A', 'Highway X'],
    'Lat': [28.538336, 28.550000, 28.560000],
    'Long': [-81.379234, -80.580000, -80.590000]
})

# Function to calculate distance between two points
def calculate_distance(lat1, lon1, lat2, lon2):
    return gd((lat1, lon1), (lat2, lon2)).km

# Find closest locations and draw lines
for _, site_row in launch_sites_df.iterrows():
    site_name = site_row['Launch Site']
    site_lat = site_row['Lat']
    site_long = site_row['Long']
    
    for location_type in locations_df['Type'].unique():
        filtered_locations = locations_df[locations_df['Type'] == location_type]
        
        min_distance = float('inf')
        closest_location = None
        closest_lat = None
        closest_long = None
        
        for _, loc_row in filtered_locations.iterrows():
            loc_lat = loc_row['Lat']
            loc_long = loc_row['Long']
            distance = calculate_distance(site_lat, site_long, loc_lat, loc_long)
            
            if distance < min_distance:
                min_distance = distance
                closest_location = loc_row['Name']
                closest_lat = loc_lat
                closest_long = loc_long
        
        # Add Marker for the closest location
        location_marker = folium.Marker(
            location=[closest_lat, closest_long],
            icon=DivIcon(
                icon_size=(20,20),
                icon_anchor=(0,0),
                html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % f"{location_type} ({min_distance:.2f} km)",
            )
        )
        site_map.add_child(location_marker)
        
        # Draw PolyLine between the launch site and the closest location
        line_coordinates = [[site_lat, site_long], [closest_lat, closest_long]]
        polyline = folium.PolyLine(
            locations=line_coordinates,
            color='blue',
            weight=2,
            opacity=0.7
        )
        site_map.add_child(polyline)

# Display the map
site_map



<class 'ModuleNotFoundError'>: No module named 'geopy'

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
